In [1]:
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns

# ==============================================================================
# 1. CARGA DE CONFIGURAÇÃO
# ==============================================================================

def find_project_root(anchor_file="conf/config.yaml"):
    """
    Sobe os diretórios a partir do notebook atual até encontrar
    a pasta onde 'conf/config.yaml' existe.
    """
    current_path = Path.cwd()
    
    # Tenta no diretório atual e sobe até o raiz do sistema
    for parent in [current_path] + list(current_path.parents):
        potential_config = parent / anchor_file
        if potential_config.exists():
            return parent
            
    raise FileNotFoundError(f"Não foi possível encontrar a raiz do projeto contendo '{anchor_file}'.")

# 1. Definir BASE_DIR (Raiz do Projeto)
try:
    BASE_DIR = find_project_root("conf/config.yaml")
    print(f"📂 Raiz do Projeto encontrada: {BASE_DIR}")
except FileNotFoundError as e:
    # Fallback manual caso a busca automática falhe (ajuste se necessário)
    print("Busca automática falhou. Usando fallback.")
    BASE_DIR = Path("/Users/lucasborges/Downloads/TCC")

# 2. Carregar o YAML da pasta conf
CONFIG_PATH = BASE_DIR / "conf/config.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# ==============================================================================
# 2. ATALHOS E VARIÁVEIS GLOBAIS
# ==============================================================================

# Atalhos dos Dicionários do YAML
P = {k: BASE_DIR / v for k, v in config['paths'].items()} # P de Paths
F = config['files']                                       # F de Files
PM = config['params']                                     # PM de Params

print(f"⚙️ Configuração carregada de: {CONFIG_PATH}")

# ==============================================================================
# 3. PONTE DE VARIÁVEIS
# ==============================================================================

# Caminhos de Arquivos (Apontando para o YAML)
TRAIN_EMB_PATH       = P['processed'] / F['track_embeddings']
NEW_EMB_PATH         = P['processed'] / F['new_track_embeddings']
X_TRAIN_PATH         = P['processed'] / F['train_features']
X_TEST_PATH          = P['processed'] / F['test_features']
MATCHING_MAP_PATH    = P.get('graphs_coarsened', P['graphs_bipartite']) / F['matching_map']
SUPER_EMB_PATH       = P.get('graphs_super', P['graphs_bipartite'])     / F['super_embeddings']

# Parâmetros
SEED                 = PM['seed']

# Configurações Visuais Padrão
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300

📂 Raiz do Projeto encontrada: /Users/lucasborges/Downloads/TCC
⚙️ Configuração carregada de: /Users/lucasborges/Downloads/TCC/conf/config.yaml


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import gc

# --- CONFIGURAÇÃO DINÂMICA (Usando P e F) ---
print("Mapeando arquivos para Inferência (Cold-Start)...")

# Inputs
X_TEST_PATH       = P['processed'] / F['test_features']
X_TRAIN_PATH      = P['processed'] / F['train_features'] # Necessário para calcular centróides
MATCHING_MAP_PATH = P['graphs_coarsened'] / F['matching_map']
SUPER_EMB_PATH    = P['graphs_super'] / F['super_embeddings']

# Output
OUT_NEW_EMB       = P['processed'] / F['new_track_embeddings']

# Validação
for path in [X_TEST_PATH, MATCHING_MAP_PATH, SUPER_EMB_PATH]:
    if not path.exists():
        print(f"Aviso: Arquivo não encontrado: {path.name}")

print(f"Output definido para: {OUT_NEW_EMB.name}")

Mapeando arquivos para Inferência (Cold-Start)...
Output definido para: new_track_embeddings_mean.parquet


In [3]:
def resolve_cold_start_weighted(df_test, df_super_feats, df_super_emb, feat_cols, batch_size=2000, k=5):
    print(f"Iniciando KNN Ponderado (K={k}) para {len(df_test):,} músicas...")

    # 1. Preparação
    df_super_feats = df_super_feats.sort_values("super_track_id").reset_index(drop=True)
    X_super = df_super_feats[feat_cols].to_numpy().astype('float32')
    
    scaler = StandardScaler()
    X_super_scaled = scaler.fit_transform(X_super)
    
    knn = NearestNeighbors(n_neighbors=k, metric='euclidean', n_jobs=-1)
    knn.fit(X_super_scaled)
    
    emb_cols = [c for c in df_super_emb.columns if c.startswith("emb_")]
    emb_lookup = df_super_feats[["super_track_id"]].merge(df_super_emb, on="super_track_id", how="left")
    emb_matrix = emb_lookup[emb_cols].fillna(0).to_numpy().astype('float32')
    
    results = []
    
    for i in tqdm(range(0, len(df_test), batch_size)):
        batch = df_test.iloc[i : i + batch_size].copy()
        X_batch = batch[feat_cols].fillna(0).to_numpy().astype('float32')
        X_batch_scaled = scaler.transform(X_batch)
        
        dists, indices = knn.kneighbors(X_batch_scaled)
        
        # --- CÁLCULO DE PESOS (Softmax Estável) ---
        # Mantendo sua lógica excelente de estabilidade numérica
        neg_dists = -dists
        max_neg_dists = np.max(neg_dists, axis=1, keepdims=True)
        exp_x = np.exp(neg_dists - max_neg_dists)
        weights = exp_x / np.sum(exp_x, axis=1, keepdims=True) # [Batch, K]
        
        # --- INTERPOLAÇÃO VETORIZADA ---
        # Multiplicação de matrizes 3D para evitar o loop j in range(k)
        # neighbor_embs shape: [Batch, K, Emb_Dim]
        neighbor_embs = emb_matrix[indices] 
        
        # Ponderação: (Batch, 1, K) @ (Batch, K, Emb_Dim) -> (Batch, 1, Emb_Dim)
        batch_embs = np.matmul(weights[:, np.newaxis, :], neighbor_embs).squeeze(1)
        
        # --- RE-NORMALIZAÇÃO L2 ---
        # Garante que a música nova tenha a mesma escala métrica que as músicas de treino
        norms = np.linalg.norm(batch_embs, axis=1, keepdims=True)
        batch_embs = batch_embs / (norms + 1e-10) # 1e-10 evita divisão por zero
        
        res_df = batch[["track_uri"]].copy()
        res_df[emb_cols] = batch_embs
        results.append(res_df)
        
    return pd.concat(results, ignore_index=True)


def get_super_features_light():
    """
    Recalcula as features médias dos super-nós da forma mais leve possível
    e deleta o X_train da memória imediatamente.
    """
    print("Recalculando features dos Super-Nós (Modo Econômico)...")
    
    # 1. Carregar Map (Track -> Super_Node)
    # Usamos MATCHING_MAP_PATH definido na célula mestra do S6.1
    map_df = pd.read_parquet(MATCHING_MAP_PATH)
    col_uri = "original_track_uri" if "original_track_uri" in map_df.columns else "track_uri"
    map_df = map_df.rename(columns={col_uri: "track_uri"})[["track_uri", "super_track_id"]]
    
    # 2. Carregar X_train (Features de Áudio originais)
    if not X_TRAIN_PATH.exists():
        raise FileNotFoundError(f"X_train não encontrado em {X_TRAIN_PATH}")
        
    df_train = pd.read_parquet(X_TRAIN_PATH)
    if "track_uri" not in df_train.columns and "id" in df_train.columns:
        df_train = df_train.rename(columns={"id": "track_uri"})
        
    # 3. Merge para associar features aos super-nós
    print(f"   Cruzando {len(df_train):,} faixas com mapa...")
    merged = df_train.merge(map_df, on="track_uri", how="inner")
    
    # Limpeza imediata de memória para evitar crash
    del df_train, map_df
    gc.collect()
    
    # Identificar colunas numéricas (features que o KNN usará)
    cols = [c for c in merged.columns if c not in ["track_uri", "super_track_id"] and merged[c].dtype.kind in 'bifc']
    
    # 4. Groupby e Mean (Cálculo do Centróide de Áudio do Super-Nó)
    print("   Calculando centróides...")
    super_feats = merged.groupby("super_track_id")[cols].mean().reset_index()
    
    del merged
    gc.collect()
    
    return super_feats, cols

In [4]:
# --- EXECUÇÃO ---

# 1. Carregar Embeddings GNN
print("Carregando Embeddings dos Super-Nós...")
df_super_emb = pd.read_parquet(SUPER_EMB_PATH)

# 2. Obter Features dos Centróides (Super-Nós)
df_super_feats, feat_cols = get_super_features_light()
print(f"Features de Super-Nós prontas: {df_super_feats.shape}")

# 3. Carregar Teste e Mapa
print("Carregando Teste e Mapa...")
df_test = pd.read_parquet(X_TEST_PATH)
if "track_uri" not in df_test.columns and "id" in df_test.columns: 
    df_test = df_test.rename(columns={"id": "track_uri"})

map_df = pd.read_parquet(MATCHING_MAP_PATH)
col_uri = "original_track_uri" if "original_track_uri" in map_df.columns else "track_uri"
map_df = map_df.rename(columns={col_uri: "track_uri"})

# 4. Separar Casos
print(" Separando casos (Lookup vs KNN)...")
# Lookup: Músicas que já cairam no grafo
lookup_ids = df_test.merge(map_df, on="track_uri", how="inner")[["track_uri", "super_track_id"]]
known_uris = set(lookup_ids["track_uri"])

# Orphans: Músicas novas
df_orphans = df_test[~df_test["track_uri"].isin(known_uris)].copy()

print(f"   Via Lookup direto: {len(lookup_ids):,}")
print(f"   Via KNN (Cold-Start): {len(df_orphans):,}")

final_parts = []

# CASO A: Lookup Direto
if len(lookup_ids) > 0:
    print("Processando Lookup Direto...")
    # Merge simples
    df_lookup = lookup_ids.merge(df_super_emb, on="super_track_id", how="inner")
    if "super_track_id" in df_lookup.columns:
        df_lookup = df_lookup.drop(columns=["super_track_id"])
    final_parts.append(df_lookup)

# CASO B: KNN Ponderado
if len(df_orphans) > 0:
    print("Processando KNN Ponderado...")
    df_knn = resolve_cold_start_weighted(
        df_orphans, 
        df_super_feats, 
        df_super_emb, 
        feat_cols, 
        batch_size=2000, 
        k=5 
    )
    final_parts.append(df_knn)

# 5. Concatenar e Salvar
if final_parts:
    df_final = pd.concat(final_parts, ignore_index=True)
    print(f"Salvando {len(df_final):,} linhas em: {OUT_NEW_EMB}")
    df_final.to_parquet(OUT_NEW_EMB, index=False)
    print("\nCONCLUÍDO! Embeddings de teste gerados (Refined Approach).")
else:
    print("Aviso: Nenhuma música processada.")

Carregando Embeddings dos Super-Nós...
Recalculando features dos Super-Nós (Modo Econômico)...
   Cruzando 324,305 faixas com mapa...
   Calculando centróides...
Features de Super-Nós prontas: (20641, 52)
Carregando Teste e Mapa...
 Separando casos (Lookup vs KNN)...
   Via Lookup direto: 0
   Via KNN (Cold-Start): 60,417
Processando KNN Ponderado...
Iniciando KNN Ponderado (K=5) para 60,417 músicas...


100%|██████████| 31/31 [00:02<00:00, 14.34it/s]


Salvando 60,417 linhas em: /Users/lucasborges/Downloads/TCC/data/processed/new_track_embeddings_mean.parquet

CONCLUÍDO! Embeddings de teste gerados (Refined Approach).


In [5]:
import numpy as np
import pandas as pd

def validate_embeddings_norm(df, name="Dataset"):
    # 1. Identificar colunas de embedding (ex: emb_mean_000...)
    emb_cols = [c for c in df.columns if c.startswith("emb_")]
    if not emb_cols:
        print(f"❌ {name}: Nenhuma coluna de embedding encontrada.")
        return
    
    # 2. Converter para numpy
    embs = df[emb_cols].to_numpy()
    
    # 3. Calcular a norma L2 de cada linha
    norms = np.linalg.norm(embs, axis=1)
    
    # 4. Estatísticas
    mean_norm = np.mean(norms)
    std_norm = np.std(norms)
    min_norm = np.min(norms)
    max_norm = np.max(norms)
    
    # 5. Verificação de Tolerância (erro de ponto flutuante)
    # Todos devem estar muito próximos de 1.0
    is_valid = np.allclose(norms, 1.0, atol=1e-3)
    
    print(f"=== Validação: {name} ===")
    print(f"Norma Média: {mean_norm:.6f} (±{std_norm:.6f})")
    print(f"Range: [{min_norm:.6f} a {max_norm:.6f}]")
    
    if is_valid:
        print("✅ SUCESSO: Todos os vetores possuem norma unitária (1.0).")
    else:
        print("⚠️ ALERTA: Detectados vetores fora da norma unitária.")
        print("   Isso pode causar viés na Similaridade de Cosseno no S7.")
    print("-" * 30)

# --- EXECUÇÃO ---
# Verificando os dois arquivos principais gerados/usados no S6.1
try:
    # Substitua pelos nomes reais dos seus DataFrames se eles ainda estiverem em memória
    # Ou carregue-os dos caminhos definidos (OUT_TRACK_EMB e o retorno da resolve_cold_start)
    
    # Exemplo com os arquivos salvos:
    df_train_val = pd.read_parquet(P['processed'] / F['track_embeddings'])
    df_test_val = pd.read_parquet(P['processed'] / F['new_track_embeddings'])
    
    validate_embeddings_norm(df_train_val, "Tracks de Treino (GraphSAGE)")
    validate_embeddings_norm(df_test_val, "Tracks Novas (Weighted k-NN)")
    
except Exception as e:
    print(f"Erro ao carregar arquivos para validação: {e}")

=== Validação: Tracks de Treino (GraphSAGE) ===
Norma Média: 1.000000 (±0.000000)
Range: [1.000000 a 1.000000]
✅ SUCESSO: Todos os vetores possuem norma unitária (1.0).
------------------------------
=== Validação: Tracks Novas (Weighted k-NN) ===
Norma Média: 1.000000 (±0.000000)
Range: [1.000000 a 1.000000]
✅ SUCESSO: Todos os vetores possuem norma unitária (1.0).
------------------------------
